# ETL for Seattle Airbnb Dataset

### Import Dependencies

In [1]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine

### Import "Listing" CSV

In [2]:
# store the csv data to a dataframe
listing_host_csv = "Data_CSV/listings.csv"
listing_host_df = pd.read_csv(listing_host_csv, encoding="utf8")
# choose encoding 'utf8' to not get errors

In [3]:
# inspect the data and the datatypes in the columns
listing_host_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 92 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3818 non-null   int64  
 1   listing_url                       3818 non-null   object 
 2   scrape_id                         3818 non-null   int64  
 3   last_scraped                      3818 non-null   object 
 4   name                              3818 non-null   object 
 5   summary                           3641 non-null   object 
 6   space                             3249 non-null   object 
 7   description                       3818 non-null   object 
 8   experiences_offered               3818 non-null   object 
 9   neighborhood_overview             2786 non-null   object 
 10  notes                             2212 non-null   object 
 11  transit                           2884 non-null   object 
 12  thumbn

In [4]:
# we need to change the name of the column "name" to something SQL doesn't recognise
listing_host_df.rename(index=str, columns={'name':'listing_name'}, inplace=True)
listing_host_df.head()

,id,listing_url,scrape_id,last_scraped,listing_name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,...,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",...,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazing water view,New modern house built in 2013. Spectacular s...,"Our house is modern, light and fresh with a wa...",New modern house built in 2013. Spectacular s...,none,Upper Queen Anne is a charming neighborhood fu...,...,10.0,f,NaN,WASHINGTON,f,strict,f,f,2,1.15
3,7421966,https://www.airbnb.com/rooms/7421966,20160104002432,2016-01-04,Queen Anne Chateau,A charming apartment that sits atop Queen Anne...,NaN,A charming apartment that sits atop Queen Anne...,none,NaN,...,NaN,f,NaN,WASHINGTON,f,flexible,f,f,1,NaN
4,278830,https://www.airbnb.com/rooms/278830,20160104002432,2016-01-04,Charming craftsman 3 bdm house,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,Cozy family craftman house in beautiful neighb...,none,We are in the beautiful neighborhood of Queen ...,...,9.0,f,NaN,WASHINGTON,f,strict,f,f,1,0.89


### Creating and Cleaning the "Listing" DataFrame

In [5]:
# create a new dataframe by selecting some columns of the dataframe "listings_host_df"
listing_df = listing_host_df[["id","listing_name","street","neighbourhood_cleansed","neighbourhood_group_cleansed","city","state","zipcode","latitude","longitude","is_location_exact","property_type","room_type","accommodates","bathrooms","bedrooms","beds","bed_type","square_feet","price","weekly_price","monthly_price","security_deposit","cleaning_fee","guests_included","extra_people","minimum_nights","maximum_nights","has_availability","availability_30","availability_60","availability_90","availability_365","number_of_reviews","first_review","last_review","review_scores_rating","review_scores_accuracy","review_scores_cleanliness","review_scores_checkin","review_scores_communication","review_scores_location","review_scores_value","requires_license","instant_bookable","cancellation_policy","require_guest_profile_picture","require_guest_phone_verification","reviews_per_month","host_id"]]

In [6]:
listing_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3818 entries, 0 to 3817
Data columns (total 50 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                3818 non-null   int64  
 1   listing_name                      3818 non-null   object 
 2   street                            3818 non-null   object 
 3   neighbourhood_cleansed            3818 non-null   object 
 4   neighbourhood_group_cleansed      3818 non-null   object 
 5   city                              3818 non-null   object 
 6   state                             3818 non-null   object 
 7   zipcode                           3811 non-null   object 
 8   latitude                          3818 non-null   float64
 9   longitude                         3818 non-null   float64
 10  is_location_exact                 3818 non-null   object 
 11  property_type                     3817 non-null   object 
 12  room_type  

In [7]:
# convert datetime columns
listing_df['first_review'] = pd.to_datetime(listing_df['first_review'])
listing_df['last_review'] = pd.to_datetime(listing_df['last_review'])

C:\Users\vivek\AppData\Local\Temp\ipykernel_19980\55908757.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listing_df['first_review'] = pd.to_datetime(listing_df['first_review'])
C:\Users\vivek\AppData\Local\Temp\ipykernel_19980\55908757.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listing_df['last_review'] = pd.to_datetime(listing_df['last_review'])


In [8]:
# replace all string values with proper boolean values True/ False
boolean_columns = ['is_location_exact', 'has_availability', 'requires_license', 'instant_bookable', 'require_guest_profile_picture', 'require_guest_phone_verification']

for i in boolean_columns:
    listing_df[i].replace({'t':True, 'f':False}, inplace=True)

C:\Users\vivek\AppData\Local\Temp\ipykernel_19980\1961470656.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listing_df[i].replace({'t':True, 'f':False}, inplace=True)


In [9]:
# convert currency values to numeric values instead of strings

# define the currency columns
currency_cols=["price","weekly_price","monthly_price","security_deposit","cleaning_fee","extra_people"]

# remove the dollar-sign and commas
listing_df[currency_cols] = listing_df[currency_cols].replace({'\$':'', ',':''}, regex=True)

# change the datatype to appropriate numeric
for i in currency_cols:
    listing_df[i] = pd.to_numeric(listing_df[i])
    
listing_df[currency_cols].info()

<class 'pandas.core.frame.DataFrame'>
Index: 3818 entries, 0 to 3817
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             3818 non-null   float64
 1   weekly_price      2009 non-null   float64
 2   monthly_price     1517 non-null   float64
 3   security_deposit  1866 non-null   float64
 4   cleaning_fee      2788 non-null   float64
 5   extra_people      3818 non-null   float64
dtypes: float64(6)
memory usage: 208.8+ KB


C:\Users\vivek\AppData\Local\Temp\ipykernel_19980\2910897604.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listing_df[currency_cols] = listing_df[currency_cols].replace({'\$':'', ',':''}, regex=True)
C:\Users\vivek\AppData\Local\Temp\ipykernel_19980\2910897604.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listing_df[i] = pd.to_numeric(listing_df[i])


## Creating and Cleaning "Hosts" dataframe

In [10]:
# Create new data with select columns for the AirBnB hosts
host_df = listing_host_df[["host_id","host_name","host_since","host_location","host_response_time","host_response_rate","host_acceptance_rate","host_is_superhost","host_neighbourhood","host_listings_count","host_has_profile_pic","host_identity_verified"]]

In [11]:
# removing duplicates
host_df.drop_duplicates(keep='first', inplace=True)

C:\Users\vivek\AppData\Local\Temp\ipykernel_19980\1857549933.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host_df.drop_duplicates(keep='first', inplace=True)


In [12]:
# convert dates to datetime format
host_df['host_since'] = pd.to_datetime(host_df['host_since'])

C:\Users\vivek\AppData\Local\Temp\ipykernel_19980\1267217527.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host_df['host_since'] = pd.to_datetime(host_df['host_since'])


In [13]:
# covert boolean cloumns to boolean
boolean_columns = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified']

for i in boolean_columns:
    host_df[i].replace({'t':True, 'f':False}, inplace=True)

C:\Users\vivek\AppData\Local\Temp\ipykernel_19980\2296379888.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host_df[i].replace({'t':True, 'f':False}, inplace=True)


In [14]:
# convert percentage columns to nueric columns
percentage_columns = ['host_response_rate', 'host_acceptance_rate']

for i in percentage_columns:
    host_df[i] = host_df[i].replace({'\$':'', ',':'', '%':''}, regex=True)
    host_df[i] = pd.to_numeric(host_df[i])
    
host_df

C:\Users\vivek\AppData\Local\Temp\ipykernel_19980\706744946.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host_df[i] = host_df[i].replace({'\$':'', ',':'', '%':''}, regex=True)
C:\Users\vivek\AppData\Local\Temp\ipykernel_19980\706744946.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  host_df[i] = pd.to_numeric(host_df[i])


,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_has_profile_pic,host_identity_verified
0,956883,Maija,2011-08-11,"Seattle, Washington, United States",within a few hours,96.0,100.0,False,Queen Anne,3.0,True,True
1,5177328,Andrea,2013-02-21,"Seattle, Washington, United States",within an hour,98.0,100.0,True,Queen Anne,6.0,True,True
2,16708587,Jill,2014-06-12,"Seattle, Washington, United States",within a few hours,67.0,100.0,False,Queen Anne,2.0,True,True
3,9851441,Emily,2013-11-06,"Seattle, Washington, United States",NaN,NaN,NaN,False,Queen Anne,1.0,True,True
4,1452570,Emily,2011-11-29,"Seattle, Washington, United States",within an hour,100.0,NaN,False,Queen Anne,2.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...
3811,11847918,Ryan,2014-02-01,"Seattle, Washington, United States",within a few hours,100.0,100.0,False,Fremont,1.0,True,True
3814,46566046,Glen,2015-10-14,"Seattle, Washington, United States",within an hour,100.0,100.0,False,Portage Bay,1.0,True,True
3815,52791370,Virginia,2015-12-30,US,NaN,NaN,NaN,False,NaN,1.0,True,False
3816,25522052,Karen,2015-01-03,"Tacoma, Washington, United States",within an hour,100.0,NaN,False,NaN,1.0,True,True


## Creating and Cleaning "Availibility" Dataframe

In [15]:
# import the required csv file
availability_file = "Data_CSV/calendar.csv"
availabilitiy_df = pd.read_csv(availability_file, encoding="utf8")

In [16]:
# inspect the dataframe
availabilitiy_df.columns

Index(['listing_id', 'date', 'available', 'price'], dtype='object')

In [17]:
# change the 'date' column name to something SQL doesn't recognise
availabilitiy_df.rename(index=str, columns={'date':'available_date'}, inplace=True)
availabilitiy_df['available_date'] = pd.to_datetime(availabilitiy_df['available_date'])

In [18]:
# convert the boolean columns to proper boolean values
availabilitiy_df['available'].replace({'t':True, 'f':False}, inplace=True)

In [19]:
# remove dollar sign and make the appropriate columns numeric
availabilitiy_df['price'] = availabilitiy_df['price'].replace({'\$':'', ',':''}, regex=True)
availabilitiy_df['price'] = pd.to_numeric(availabilitiy_df['price'])
availabilitiy_df.head()

,listing_id,available_date,available,price
0,241032,2016-01-04,True,85.0
1,241032,2016-01-05,True,85.0
2,241032,2016-01-06,False,NaN
3,241032,2016-01-07,False,NaN
4,241032,2016-01-08,False,NaN


## Creating and Cleaning "Reviews" DataFrame

In [20]:
# store the csv file in a DataFrame
reviews_file = "Data_CSV/reviews.csv"
reviews_df = pd.read_csv(reviews_file, encoding="utf8")
reviews_df.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [21]:
# rename columns to non-sql name
reviews_df.rename(columns={'id': 'review_id', 'date': 'review_date'}, inplace=True)

In [22]:
# selecting columns that are required
reviews_df = reviews_df[['review_id', "listing_id", "review_date", "reviewer_id", "reviewer_name", "comments"]]

In [23]:
# convert date columns to datetime
reviews_df['review_date'] = pd.to_datetime(reviews_df['review_date'])
reviews_df.head()

,review_id,listing_id,review_date,reviewer_id,reviewer_name,comments
0,38917982,7202016,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,39087409,7202016,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,39820030,7202016,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,40813543,7202016,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,41986501,7202016,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


# Load the Data to SQL

In [24]:
# establish connection with MySQL database and create the engine
user = 'root'
password = '1234'
host = '127.0.0.1'
port = 3306
database = 'seattle_airbnb_db'

engine = create_engine(
    url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
        user, password, host, port, database
        )
    )

In [25]:
# checking if the table is created in the database
# engine.table_names()

In [26]:
# add the DataFrames to the SQL Database
host_df.to_sql(name="airbnb_hosts",con=engine,if_exists="append",index=False,chunksize=2000)
listing_df.to_sql(name="listings",con=engine,if_exists="append",index=False,chunksize=200)
availabilitiy_df.to_sql(name="property_availability",con=engine,if_exists="append",index=False,chunksize=2000)
reviews_df.to_sql(name="property_reviews",con=engine,if_exists="append",index=False,chunksize=2000)

84849